## **Ingesta del archivo `drivers.json`**

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../utils/common_functions"

### Paso 1 - Leer el archivo JSON

In [ ]:
from pyspark.sql.functions import col, concat, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [ ]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
  
])

In [ ]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)  
])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
drivers_df = spark.read \
.schema(drivers_schema) \
.json(f"{raw_folder_path}/drivers.json")
#.json(f"{raw_folder_path}/{v_file_date}/drivers.json")

### Paso 2 - Renombrar columnas y añadir nuevas columnas

In [ ]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                    .withColumn("data_source", lit(v_data_source)) \
                                    .withColumn("file_date", lit(v_file_date))

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
drivers_with_ingestion_date_df = add_ingestion_date(drivers_with_columns_df)

### Paso 3 - Eliminar las columnas no deseadas

In [ ]:
drivers_final_df = drivers_with_ingestion_date_df.drop(col("url"))

### Paso 4 - Escribir datos en el datalake como parquet y crear la tabla **drivers** en la base de datos **f1_processed**

In [ ]:
# Recordar que Databricks utiliza Hive para la gestión de tablas
# Esto quiere decir que una tabla en Hive se asocia con un directorio específico que contiene archivos de datos
# Aunque los datos subyacentes son solo archivos en un sistema de archivos, Hive proporciona una interfaz que presenta 
# estos datos como tablas.
# La base de datos tambien es un directorio. Podemos crearla manualmente, como con lenguaje SQL.
# Hive permite a los usuarios interactuar con esos datos utilizando un lenguaje similar a SQL
# En resumen, solo se esta creando un directorio y un archivo
# Escribimos el archivo con formato PARQUET en la base de datos "f1_processed" y en la tabla "drivers"
drivers_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.drivers")

In [ ]:
dbutils.notebook.exit("Success")